# CS486 - Artificial Intelligence
## Lesson 4 - Heuristics

Efficient search often comes down to finding effective heuristics for your problem. In this lesson, we'll explore various heuristics for a classic problem: Cannibals and Missionaries. First, let's set up our notebook:

In [ ]:
from helpers import *
from aima.search import *

## Cannibals & Missionaries

## insert image here

In the Cannibals and Missionaries problem you have $n$ cannibals and $n$ missionaries trying to cross a river on a single boat that can carry up to two people at a time. You must come up with a strategy for moving everyone across the river while making sure that the cannibals never outnumber the missionaries on either bank. 

In this notebook, we set $n=3$, which means we are moving 3 missionaries and 3 cannibals across the river. Let's revisit the `Problem` class to remind ourselves how a search problem is constructed:

In [ ]:
%pdoc Problem

First, we need to encode a state. We can encode every state in a 6-tuple that contains:

* The number of **cannibals** on the **left** bank
* The number of **missionaries** on the **left** bank
* The status of the **boat** on the **left** bank: A `1` if the boat is present, a `0` otherwise. 
* The number of **cannibals** on the **right** bank
* The number of **missionaries** on the **right** bank
* The status of the **boat** on the **right** bank: A `1` if the boat is present, a `0` otherwise. 

The initial state is `(3,3,1,0,0,0)` and the goal state is `(0,0,0,3,3,3)`. 

Next, we need to define **actions**. Actions are encoded as the difference to be applied to a tuple. For instance, starting at the initial state, moving 1 missionary and 1 cannibal from the left bank to the right would be encoded as:

```
(-2,0,-1,2,0,1)
```

Note that the boat also moved sides. Below is an implementation of this problem as a search problem:

In [ ]:
class Cannibals(Problem):    
    def actions(self, state):
        # generate all possible combinations of moving two people
        return [(m*(state[-1] - state[2]), c*(state[-1]-state[2]), # set sign (+/-) for m & c, depending on which side the boat is on
                -1 if state[2] == 1 else 1,                        # update boat location
                m*(state[2] - state[-1]), c*(state[2]-state[-1]),  # set sign (+/-) for m & c, depending on which side the boat is on
                -1 if state[-1] == 1 else 1)                       # update boat location
                for m in range(0, 3)                               # up to 2 missionaries
                for c in range(0, 3)                               # up to 2 cannibals
                if 1 <= m+c <= 2]                                  # min/max boat capacity is 1 and 2, respectively
    
    def result(self, state, action):
        res = tuple([state[i] + a for i, a in enumerate(action)])
        # valid results must maintain the following invariants:
        # 1. number of m/c on a side >= 0 and <= 3
        # 2. for each side, m == 0 or m > c
        return (res if all(map(lambda x: 0 <= x <= 3, res)) and (res[0] == 0 or res[0] >= res[1]) and (res[-3] == 0 or res[-3] >= res[-2]) 
                else state)
    
    # We'll instrument the goal test with a counter
    @counter
    def goal_test(self, state):
        return state == self.goal # (0, 0, 0, 3, 3, 1)

## Heuristics

So now that we've constructed the problem, let's see what an uninformed search would yield: 

In [ ]:
searches = [
    breadth_first_graph_search,
    depth_first_graph_search
]

for search in searches:
    problem = Cannibals(initial=(3,3,1,0,0,0),goal=(0, 0, 0, 3, 3, 1))
    result = search(problem)
    print("{:26} {:^11}".format(search.__name__,problem.goal_test.count))

Now let's see if we can devise any heuristics to improve uninformed search.

In [ ]:
def heuristic(node):
    return 0

problem = Cannibals(initial=(3,3,1,0,0,0),goal=(0, 0, 0, 3, 3, 1))
astar_search(problem,heuristic)
problem.goal_test.count